In [ ]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import cartopy.feature as cfeature
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import figure
# import metpy.calc as mpcalc
import matplotlib.ticker as ticker
from matplotlib.ticker import ScalarFormatter
import xarray as xr
import os
from matplotlib.colors import LogNorm
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle
# from mpl_toolkits.basemap import Basemap
import pandas as pd
from dask.diagnostics import ProgressBar
import matplotlib.cm as cm
import datetime as dt
%xmode Plain

##### Reading in data

In [ ]:
qrvE = xr.open_dataset('/gws/nopw/j04/kscale/USERS/gyang/wave_data/ERA5_qrv_28plev/qvr_east_coefficient_20160801_k2-40_p2-30_28plev_era5_from_2016.nc')
qrvE

In [ ]:
Dn = xr.open_dataset('/gws/nopw/j04/kscale/USERS/gyang/wave_data/ERA5_qrv_28plev/Dn.nc')
Dn

In [ ]:
# DR_tm_CTC5RAL = xr.open_dataset('/gws/nopw/j04/kscale/USERS/emg/data/LoSSETT_out/channel_n2560_RAL3p2/DRdir2dt_Nlmax5_channel_n2560_RAL3p2_DS_tmean.nc')
# DRI_tm = DR_tm_CTC5RAL['LoSSET_DR']

# DR_tm_CTC5GAL = xr.open_dataset('/gws/nopw/j04/kscale/USERS/emg/data/LoSSETT_out/channel_n2560_GAL9/DRdir2dt_Nlmax5_channel_n2560_GAL9_DS_tmean.nc')
# DRI_tm = DR_tm_CTC5GAL['LoSSET_DR']

# DR_tm_ERA5 = xr.open_dataset('/gws/nopw/j04/kscale/USERS/emg/data/LoSSETT_out/ERA5_0p5deg_3h/DRdir2dt_Nlmax5_ERA5_DYAMOND_Summer_tmean.nc')
# DRI_tm = DR_tm_ERA5['LoSSET_DR']

In [ ]:
# model = 'kscaleGAL9n2560'
# directory = "/gws/nopw/j04/kscale/USERS/emg/data/LoSSETT_out/channel_n2560_GAL9/"
# dates = pd.date_range(start="2016-08-01", end="2016-09-09").strftime('%Y%m%d')
# file_paths = [f"{directory}/DRdir2dt_Nlmax5_{model}_0p5_DS_{date}.nc" for date in dates]
# ds = xr.open_mfdataset(file_paths, combine='by_coords',engine='netcdf4')
model = 'kscaleRAL3n2560'
directory = "/gws/nopw/j04/kscale/USERS/emg/data/LoSSETT_out/channel_n2560_RAL3p2/"
dates = pd.date_range(start="2016-08-01", end="2016-09-09").strftime('%Y%m%d')
file_paths = [f"{directory}/DRdir2dt_Nlmax5_{model}_0p5_DS_{date}.nc" for date in dates]
ds = xr.open_mfdataset(file_paths, combine='by_coords',engine='netcdf4')
# ds
# directory = "/gws/nopw/j04/kscale/USERS/emg/data/LoSSETT_out/ERA5_0p5deg_3h"
# dates = pd.date_range(start="2016-08-01", end="2016-09-09").strftime('%Y-%m-%d')
# file_paths = [f"{directory}/DRdir2dt_Nlmax5_era5_{date}.nc" for date in dates]
# ds = xr.open_mfdataset(file_paths, combine='by_coords',engine='netcdf4')

In [ ]:
KW_CTC5RAL_DS = xr.open_dataset('/gws/nopw/j04/kscale/USERS/gyang/wave_data/analysis_forecast_80days/outdir_20160801T0000Z/DMn1280GAL9/channel_n2560_RAL3p2/uz_Kelvin_k2-40_p2-30_analysis_channel_n2560_RAL3p2.nc')
# R1_CTC5RAL_DS = xr.open_dataset('/gws/nopw/j04/kscale/USERS/gyang/wave_data/analysis_forecast_80days/outdir_20160801T0000Z/DMn1280GAL9/channel_n2560_RAL3p2/uvz_R1_kn2-40_p2-30_analysis_channel_n2560_RAL3p2.nc')
# WMRG_CTC5RAL_DS = xr.open_dataset('/gws/nopw/j04/kscale/USERS/gyang/wave_data/analysis_forecast_80days/outdir_20160801T0000Z/DMn1280GAL9/channel_n2560_RAL3p2/uvz_WMRG_kn2-40_p2-30_analysis_channel_n2560_RAL3p2.nc')
# KW_ERA5 = xr.open_dataset("/gws/nopw/j04/kscale/USERS/emg/data/DYAMOND_Summer/uz_Kelvin_ERA5.nc")
# KW_CTC5GAL_DS = xr.open_dataset('/gws/nopw/j04/kscale/USERS/gyang/wave_data/analysis_forecast_80days/outdir_20160801T0000Z/DMn1280GAL9/channel_n2560_GAL9/uz_Kelvin_k2-40_p2-30_analysis_channel_n2560_GAL9.nc')
# KW_ERA5_DS = xr.open_dataset('/gws/nopw/j04/ncas_climate_vol1/users/bc917929/wave/div200_Kelv_rm_clim_6h_kn2-40_p2-30_1998-2017.nc')
# div200KW_DS_ERA5 = KW_ERA5_DS['divergence_of_wind'].sel(t=slice('2016-08-01','2016-09-09'))
# div200KW_DS_ERA5_hov = div200KW_DS_ERA5.sel(lat=slice(5,-5)).mean(dim=['lat'])
# div200KW_DS_ERA5_hov

In [ ]:
# uKW_e5 = KW_ERA5['eastward_wind']
# uKW_ERA5
# uKW = KW_CTC5RAL_DS['u_Kelvin']
# # uKW_hov = uKW.sel(lat=lr,p=lvl)

dx = np.deg2rad(np.gradient(KW_CTC5RAL_DS['lon']))*111000
du_dx = KW_CTC5RAL_DS['u_Kelvin'].differentiate('lon') / dx
# div_1t_lonslice = du_dx.sel(t='2016-08-02T00:00',lat=slice(5,-5)).mean(dim=['lat'])
# div_1t_lonslice
div_3D = du_dx.sel(lat=slice(5,-5))
div_slice = div_3D.mean(dim=['lat'])
# div_hov.min()
div_slice

# div_lev3D = du_dx.sel(lat=slice(5,-5))
# div_lev = div_lev3D.mean(dim=['lat'])

# uKW_slice = KW_CTC5RAL_DS['u_Kelvin'].sel(t='2016-08-02T00:00',lat=slice(5,-5)).mean(dim=['lat'])
# # 
# div_lev1t = div_lev.sel(t='2016-08-02T00:00')
# div_hov

# vR1 = R1_CTC5RAL_DS['v_R1']
# vR1_hov_200 = vR1.sel(lat=slice(13,3),p=200).mean(dim=['lat'])
# vR1_hov_200
# uWMRG = WMRG_CTC5RAL_DS['u_WMRG']
# vWMRG = WMRG_CTC5RAL_DS['v_WMRG']
# lat = np.deg2rad(vWMRG['lat'].values)  # Shape (97,)
# lon = np.deg2rad(vWMRG['lon'].values)  # Shape (720,)
# R = 6371000  
# dlat = np.gradient(lat) * R  # Shape (97,)
# dlon = np.gradient(lon) * R * np.cos(lat[:, np.newaxis])  # Shape (97, 720)
# dlat = dlat[:, np.newaxis] * np.ones((1, 720))  # Shape becomes (97, 720)
# du_dx = uWMRG.differentiate('lon') / dlon
# dv_dy = vWMRG.differentiate('lat') / dlat
# div_WMRG = dv_dy
# div_WMRG_hov = div_WMRG.sel(p=200,lat=slice(5,-5)).mean(dim=['lat'])
# div_WMRG_hov.min()

In [ ]:
kscale_levs = [100,150,200,250,300,400,500,600,700,850,925,1000]

dates = [dt.datetime(2016,8,1) + dt.timedelta(i) for i in range(40)]
# for lev in kscale_levs:
    # print(f"{lev}")
ds_uvw = []
for lev in kscale_levs:
    _ds_uvw = xr.open_mfdataset(
        f"/gws/nopw/j04/kscale/DATA/outdir_20160801T0000Z/DMn1280GAL9/channel_n2560_GAL9/profile_{lev}/{date.year:04d}{date.month:02d}{date.day:02d}_20160801T0000Z_channel_profile_3hourly_{lev}_05deg.nc" for date in dates
    )
    ds_uvw.append(_ds_uvw)
ds_uvw = xr.concat(ds_uvw,dim="pressure")
ds_uvw

In [ ]:
ds_wERA5_tm = xr.open_dataset('/gws/nopw/j04/kscale/USERS/dship/ERA5/3hourly/era5_w_tmean_DS_0p5deg.nc')
ds_TERA5_tm = xr.open_dataset('/gws/nopw/j04/kscale/USERS/dship/ERA5/3hourly/era5_temptmean_DS_0p5deg.nc')
ds_vERA5_tm = xr.open_dataset('/gws/nopw/j04/kscale/USERS/dship/ERA5/3hourly/era5_v_tmean_DS_0p5deg.nc')
ds_uERA5_tm = xr.open_dataset('/gws/nopw/j04/kscale/USERS/dship/ERA5/3hourly/era5_u_tmean_DS_0p5deg.nc')
omega=ds_wERA5_tm['w']
p=ds_wERA5_tm['pressure_level']
temp = ds_TERA5_tm['t']
rgas = 287.05
g    = 9.81
rho = ((p*100) / (rgas * temp))
# rho = rho.transpose('time','level','latitude','longitude')
# rho
# # print(omega)
# # Calculate w from omega for ERA5
w = np.divide(-omega,(rho*g))
# w
w_vertslice_2D_lat = w.isel(valid_time=0).mean(dim=['lon'])
w_vertslice_2D_lat
v_vertslice_2D_lat = ds_vERA5_tm['v'].mean(dim=['lon'])
u_vertslice_2D_lat = ds_uERA5_tm['u'].mean(dim=['lon'])
# v_vertslice_2D_lat
# ds_wERA5_tm['w'].min()

In [ ]:
ds_uERA5 = xr.open_dataset('/gws/nopw/j04/kscale/USERS/dship/ERA5/3hourly/era5_u_AugSep_3h_0p5deg.nc')
uERA5_DS_tm = ds_uERA5['u'].sel(valid_time=slice('2016-08-01T01:00','2016-09-09T22:00')).mean(dim=['valid_time'])
uERA5_DS_tm.to_netcdf('/gws/nopw/j04/kscale/USERS/dship/ERA5/3hourly/era5_u_tmean_DS_0p5deg.nc')

In [ ]:
ds_uvw['upward_air_velocity'].to_netcdf('/gws/nopw/j04/kscale/USERS/emg/data/w_DS_3D_CTC5GAL.nc')
ds_uvw['y_wind'].to_netcdf('/gws/nopw/j04/kscale/USERS/emg/data/v_DS_3D_CTC5GAL.nc')
ds_uvw['x_wind'].to_netcdf('/gws/nopw/j04/kscale/USERS/emg/data/u_DS_3D_CTC5GAL.nc')

In [ ]:
ds_w=xr.open_dataset('/gws/nopw/j04/kscale/USERS/emg/data/w_DS_3D_CTC5GAL.nc')
ds_v=xr.open_dataset('/gws/nopw/j04/kscale/USERS/emg/data/v_DS_3D_CTC5GAL.nc')
ds_u=xr.open_dataset('/gws/nopw/j04/kscale/USERS/emg/data/u_DS_3D_CTC5GAL.nc')

##### Plotting of maps of tropical $\mathcal{D}_\ell(\mathbf{u})$ - time mean

In [ ]:
# plt.figure(figsize=(18,3))
# ax = plt.axes(projection=ccrs.PlateCarree())

# lon = w200t1['longitude']
# lat = w200t1['latitude']
# ax.set_extent([180,-180,-15,15],crs=ccrs.PlateCarree())
# plt.contour(lon, lat, -DRI_l220_200hPa.T, colors='k', levels=np.arange(-1,1,1e-1),extend='both')
# plt.pcolormesh(lon,lat,wmean,cmap='PiYG',alpha=0.5,vmin=-0.01,vmax=0.01)
# ax.coastlines()
# cbar=plt.colorbar(orientation='vertical', shrink=0.5, pad=0.025)
# cbar.set_label(r'$\mathcal{D}_\ell(\mathbf{u})$ (m$^2$ s$^{-3}$)')
# ax.axhline(y=0,color='k',linewidth=0.5,linestyle='--')
# cbar.formatter = ticker.ScalarFormatter()
# cbar.formatter.set_scientific(True)
# cbar.formatter.set_powerlimits((-2, 3))
# cbar.update_ticks() 
# cbar.set_ticks([-5e-3,0,5e-3])

##### Hovmollers

In [ ]:
nl = 3
lr = slice(-5,5)
lvl = 700

# DR = ds['LoSSET_DR']

# DR_3D = DR.sel(n_scales=nl,latitude=lr,level=lvl)
DR_hov = ds['LoSSET_DR'].sel(level=lvl,latitude=lr,n_scales=nl).mean(dim=['latitude'])
DR_hov

In [ ]:
# Transform ERA5 data to -180 to 180 before plotting
# nm = mcolors.SymLogNorm(linthresh=1, linscale=1, vmin=-10, vmax=10)
fig,ax = plt.subplots(figsize=(4,6))
lon = ds['longitude']
t = ds['time']
twave = div_hov['t']
lonwave = div_hov['lon']
plt.pcolormesh(lon,t,-DR_hov.T,cmap='bwr',vmin=-0.5,vmax=0.5)#,levels=np.linspace(-0.5,0.5,101),extend='both')
# plt.xticks([])
CS = plt.colorbar(shrink=0.5)
# CS.set_ticks([-10,0,10])
CS.set_label(r'$\mathcal{D}_\ell(\mathbf{u})$ (m$^2$ s$^{-3}$)')
plt.title(r'$\mathcal{D}_\ell(\mathbf{u})$ 5N-5S | 700hPa | $\ell=220$km' '\n' 'DS CTC5GAL + div$^{KW}_{700}$ c.i. 1e-4 s$^{-1}$')
# plt.contour(lon,t,div_WMRG.sel(p=200),colors='k',linewidths=1)
plt.contour(lonwave,twave,div_hov,colors='k',levels=[-4e-4,-3e-4,-2e-4,-1e-4,1e-4,2e-4,3e-4,4e-4],linewidths=1)
# plt.contour(lon,t,div_hov,colors='k',levels=[0],linewidths=0.5)

plt.xlim([45,180])
plt.ylim([dt.datetime(2016,8,1),dt.datetime(2016,8,15)])

# Add a small map as an inset
box = ax.get_position()
inset_height = 0.2  # Height of the inset map as a fraction of figure height
inset_bottom = box.y0 - inset_height + 0.05 # Add a small gap
inset_ax = fig.add_axes([box.x0, inset_bottom, box.width, inset_height], projection=ccrs.PlateCarree())
inset_ax.set_extent([45,180, -15, 15], crs=ccrs.PlateCarree())

import cartopy.mpl.ticker as cticker
# # Add land with some coloring
inset_ax.add_feature(cfeature.LAND, facecolor='lightgreen')
inset_ax.add_feature(cfeature.COASTLINE)
inset_ax.set_xticks([45,90,135,180], crs=ccrs.PlateCarree())
inset_ax.axhline(0,color='k',linewidth=0.5,linestyle='--')

lat_min, lat_max = -5, 5
lon_min, lon_max = -180, 180

# # Create a translucent rectangle
patch = Rectangle(
    (lon_min, lat_min),        # Bottom-left corner (longitude, latitude)
    lon_max - lon_min,         # Width (span in longitude)
    lat_max - lat_min,         # Height (span in latitude)
    transform=ccrs.PlateCarree(),
    color='k',
    alpha=0.3                   # Set the transparency level (0 is fully transparent, 1 is opaque)
)
inset_ax.add_patch(patch)
# inset_ax.set_xticklabels(['180°W', '120°W', '60°W', '0°', '60°E', '120°E', '180°E'])
inset_ax.set_yticks([])
plt.savefig('/home/users/emg97/emgPlots/LO_Dlu_hov_700hPa_5NS_l220km_CTC5GAL_DS_1st2w_45E180E_divKW700.png',dpi=600,bbox_inches='tight')

In [ ]:
levl = 200

DR_map = DR_tm.sel(level=levl,n_scales=3)

plt.figure(figsize=(18,6))
ax = plt.axes(projection=ccrs.PlateCarree())
lon = ds['longitude']
lat = ds['latitude']
ax.set_extent([180,-180,-15,15],crs=ccrs.PlateCarree())
plt.contourf(lon,lat,-DR_map.T,cmap='bwr',levels=np.arange(-0.5,0.5,5e-3),extend='both')
ax.coastlines()
plt.title(r'$\mathcal{D}_\ell(\mathbf{u})$' + f' at {levl}hPa ' + r'(m$^2$ s$^{-3}$) for $\ell = 220$ km | DYAMOND Summer | N2560 param')
cbar=plt.colorbar(orientation='vertical', shrink=0.25, pad=0.025)
# cbar.set_label(r'$\mathcal{D}_\ell(\mathbf{u})$ (m$^2$ s$^{-3}$)')
ax.axhline(y=0,color='k',linewidth=0.5,linestyle='--')
cbar.formatter = ticker.ScalarFormatter()
cbar.formatter.set_scientific(True)
cbar.formatter.set_powerlimits((-2, 3))
cbar.update_ticks() 
cbar.set_ticks([-0.5,0,0.5])

plt.savefig(f'/home/users/emg97/emgPlots/LO_tm_DS_200hPa_{model}_l220_15NS.png',dpi=600)

##### Animation of tropical $\mathcal{D}_\ell(\mathbf{u})$

In [ ]:
# Map
DR_anim = ds['LoSSET_DR'].sel(level=200,n_scales=0)

for i in range(len(time)):
    plt.figure(figsize=(18,3))
    ax = plt.axes(projection=ccrs.PlateCarree())

    data_at_time = DR_anim.isel(time=i)

    lon = data_at_time['longitude']
    lat = data_at_time['latitude']
    ax.set_extent([180,-180,-15,15],crs=ccrs.PlateCarree())
    plt.contourf(lon, lat, data_at_time.T, cmap='bwr', levels=np.arange(-5e-3,5e-3,5e-5),extend='both')
    ax.coastlines()
    cbar=plt.colorbar(orientation='vertical', shrink=0.5, pad=0.025)
    cbar.set_label(r'$\mathcal{D}_\ell(\mathbf{u})$ (m$^2$ s$^{-3}$)')
    ax.axhline(y=0,color='k',linewidth=0.5,linestyle='--')
    cbar.formatter = ticker.ScalarFormatter()
    cbar.formatter.set_scientific(True)
    cbar.formatter.set_powerlimits((-2, 3))
    cbar.update_ticks() 
    cbar.set_ticks([-5e-3,0,5e-3])
    
    file_number = str(i).zfill(3)

    plt.title(f'L = 55 km | DYAMOND Summer | 200hPa | 5km expl | Day: {i//8}')

    plt.savefig(f'/home/users/emg97/emgPlots/frames_LO/frame_{file_number}.png')
    plt.close()

In [ ]:
# Slice


# for i in range(len(time)):
#     plt.figure(figsize=(8,4))
#     data_at_time = DR_vertslice_2D_lat.isel(time=i)
#     if isinstance(data_at_time, xr.Dataset):
#         data_at_time = data_at_time.to_array().squeeze()  # Converts to 2D array if it's still 3D
#     elif data_at_time.ndim > 2:
#         data_at_time = data_at_time.squeeze()  # Squeeze dimensions if more than 2D
#     CS = plt.contourf(lat,lev,data_at_time.T,cmap='bwr',extend='both',levels=np.linspace(-5e-4,5e-4,101))
#     plt.gca().invert_yaxis() # The data is sorted by level index (index 12 = 1000hPa)
#     plt.ylabel('Height (hPa)')
#     # plt.title(f'L = 55 km | DYAMOND Summer | 200hPa | 5km conv | Day: {i//8}')
#     plt.xlim([-15,15])
#     cbar = plt.colorbar(CS)
#     cbar.set_label(r'Longitudianlly averaged $D_\ell(\mathbf{u})$ (m$^2$s$^{-3}$)')
#     cbar.formatter = ticker.ScalarFormatter()
#     cbar.formatter.set_scientific(True)
#     cbar.formatter.set_powerlimits((-3, 3))
#     cbar.update_ticks()
#     cbar.set_ticks([-5e-4,0,5e-4])

#     # # Add a small map as an inset
#     # box = ax.get_position()
#     # inset_height = 0.2  # Height of the inset map as a fraction of figure height
#     # inset_bottom = box.y0 - inset_height + 0.0225  # Add a small gap
#     # inset_ax = fig.add_axes([box.x0, inset_bottom, box.width, inset_height], projection=ccrs.PlateCarree())
#     # inset_ax.set_extent([-180, 180, -15, 15], crs=ccrs.PlateCarree())

#     # import cartopy.mpl.ticker as cticker
#     # # Add land with some coloring
#     # inset_ax.add_feature(cfeature.LAND, facecolor='lightgreen')
#     # inset_ax.add_feature(cfeature.COASTLINE)
#     # inset_ax.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
#     # inset_ax.set_xticklabels(['180°W', '120°W', '60°W', '0°', '60°E', '120°E', '180°E'])
#     # inset_ax.set_yticks([])

#     file_number = str(i).zfill(3)

#     plt.title(f'L = 55 km | DYAMOND Summer | 200hPa | 5km conv | Day: {i//8}')
 
#     plt.savefig(f'/home/users/emg97/emgPlots/frames_LO/frame_{file_number}.png')
#     plt.close()

##### Vertical Slices

In [ ]:
DR_vertslice_long = ds['LoSSET_DR'].sel(latitude=slice(-5,5),n_scales=3).mean(dim=['latitude'])
DR_vertslice_long
# DR_vertslice_2D_lat = DRI_tm.sel(n_scales=3,latitude=slice(-25,25)).mean(dim=['longitude'])
# DR_vertslice_2D_lat

In [ ]:
# w_tm = ds_w['upward_air_velocity'].mean(dim='time').compute()
# v_tm = ds_v['y_wind'].mean(dim=['time'])
# u_tm = ds_u['x_wind'].mean(dim=['time'])

# w_vertslice_2D_lon = w_tm.mean(dim='latitude')
# w_vertslice_2D_lat = wERA5_DS_tm.mean(dim=['lon'])

# v_vertslice_2D_lon = v_tm.mean(dim='latitude')
# v_vertslice_2D_lat = v_tm.mean(dim='longitude')
# u_vertslice_2D_lat = u_tm.mean(dim=['longitude'])


# v_vertslice_2D_lat
DR_vertslice_long.time

In [ ]:
output_dir = '/home/users/emg97/emgPlots/frames_Dlu/DS/KW_MC'
os.makedirs(output_dir, exist_ok=True)

lon=ds['longitude']
lev=ds['level']
lonwave = KW_CTC5RAL_DS['lon']
levwave = KW_CTC5RAL_DS['p']

for i in range(len(DR_vertslice_long.time)):
    fig,ax=plt.subplots(figsize=(15,5))
    DR_at_time = DR_vertslice_long.isel(time=i)
    div_at_time = div_slice.isel(t=i)
    CS = plt.contourf(lon,lev,-DR_at_time.T,cmap='bwr',extend='both',levels=np.linspace(-0.5,0.5,101))
    plt.contour(lonwave,levwave,div_at_time,colors='k',levels=[2e-4,4e-4,6e-4,8e-4],linewidths=1)
    plt.contour(lonwave,levwave,div_at_time,colors='k',levels=[-8e-4,-6e-4,-4e-4,-2e-4],linewidths=1,linestyles=':')
    plt.contour(lonwave,levwave,div_at_time,colors='k',levels=[0])
    plt.gca().invert_yaxis() # The data is sorted by level index (index 12 = 1000hPa)
    # ax.set_xlim([-15,15])
    plt.ylabel('Pressure (hPa)')
    # ax.set_yscale('log')
    plt.xticks([])
    plt.xlabel(r'Longitude ($^\circ$E)')
    plt.xlim([45,180])
    plt.ylim([925,100])
    plt.title(r'$\ell$ = 220 km | CTC5GAL DS | div$_{KW}$ c.i. 2 s$^{-1}$ | ' f'Simulation day: {i//8}')
    ax.set_yscale('log')
    pressure_levels = [925,850, 700, 500, 400, 300, 250, 200, 150, 100]
    ax.set_yticks(pressure_levels)
    ax.set_yticklabels([f"{level}" for level in pressure_levels])
    ax.yaxis.set_minor_locator(ticker.NullLocator())

    cbar = plt.colorbar(CS)
    cbar.set_label(r'5N-5S averaged $D_\ell(\mathbf{u})$ (m$^2$s$^{-3}$)')
    # cbar.set_ticks(np.linspace(-5e-5,5e-5,11))
    cbar.formatter = ticker.ScalarFormatter()
    cbar.formatter.set_scientific(True)
    cbar.formatter.set_powerlimits((-3, 3))
    cbar.update_ticks()
    cbar.set_ticks([-0.5,0,0.5])

    # Add a small map as an inset
    box = ax.get_position()
    inset_height = 0.2  # Height of the inset map as a fraction of figure height
    inset_bottom = box.y0 - inset_height + 0.02  # Add a small gap
    inset_ax = fig.add_axes([box.x0, inset_bottom, box.width, inset_height], projection=ccrs.PlateCarree())
    inset_ax.set_extent([45, 180, -5, 5], crs=ccrs.PlateCarree())
    
    import cartopy.mpl.ticker as cticker
    # Add land with some coloring
    inset_ax.add_feature(cfeature.LAND, facecolor='lightgreen')
    inset_ax.add_feature(cfeature.COASTLINE)
    inset_ax.set_xticks([45,90,135,180], crs=ccrs.PlateCarree())
    # inset_ax.set_xticklabels(['180°W', '120°W', '60°W', '0°', '60°E', '120°E', '180°E'])

    lat_min, lat_max = -5, 5
    lon_min, lon_max = 45, 180


    file_number = str(i).zfill(3)
    inset_ax.set_yticks([])
    plt.savefig(os.path.join(output_dir,f'frame_{file_number}.png'),bbox_inches='tight',dpi=300)
    plt.close()

In [ ]:
rE = 6374000
mpdeg = np.pi * rE * np.cos(np.deg2rad(0)) / 180
conv_factor_KScale = (mpdeg * 0.5) / (6 * 3600) # Lon res = 0.5 deg; time res = 6h
# print('conversion factor for K-Scale 0.5deg 6hrly data: ',conv_factor_KScale)
dominant_phasespeed = conv_factor_KScale * np.tan(np.deg2rad(75))
dominant_phasespeed

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import rotate

longitude = np.linspace(-180, 180, 360)
time = np.linspace(0, 24, 100)  # e.g., 24 hours of data
data = np.sin(2 * np.pi * (longitude[np.newaxis, :] / 360 + time[:, np.newaxis] / 24))

ang = 75
# Rotate the data by 30 degrees
rotated_data = rotate(data, angle=ang, reshape=False, mode='nearest')

# Plot original and rotated data
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

ax[0].contourf(data, levels=100,cmap='bwr')
ax[0].set_title("Original Data")
ax[0].set_xlabel("Longitude")
ax[0].set_ylabel("Time (Hours)")

ax[1].contourf(rotated_data, levels=100,cmap='bwr')
ax[1].set_title(f"Rotated Data ({ang}°/{dominant_phasespeed:.2f})" r"ms$^{-1}")
ax[1].set_xlabel("Longitude")
ax[1].set_ylabel("Time (Hours)")

plt.show()


In [ ]:

# # Create a translucent rectangle
# patch = Rectangle(
#     (lon_min, lat_min),        # Bottom-left corner (longitude, latitude)
#     lon_max - lon_min,         # Width (span in longitude)
#     lat_max - lat_min,         # Height (span in latitude)
#     transform=ccrs.PlateCarree(),
#     color='k',
#     alpha=0.3                   # Set the transparency level (0 is fully transparent, 1 is opaque)
# )
# inset_ax.add_patch(patch)
                             
# # Indicator of filter length
# lon_start = 160  # Adjust this longitude position as needed
# lat_start = 850  # Latitude of the line's starting point
# plt.plot([lon_start, lon_start + 2], [lat_start, lat_start], color='k', lw=1)
# label_text = r"$\ell$"
# label_x = lon_start + 1  # Position the label at the midpoint of the line
# label_y = lat_start + 0.2  # Adjust this as needed to position the label above the line
# plt.text(label_x, label_y, label_text, color='k', fontsize=12, ha='center', va='bottom')
# serif_length = 10  # Length of the serifs
# plt.plot([lon_start, lon_start], [lat_start - serif_length, lat_start + serif_length], color='k', lw=1)  # Left serif
# plt.plot([lon_start + 2, lon_start + 2], [lat_start - serif_length, lat_start + serif_length], color='k', lw=1)  # Right serif

In [ ]:
norm = mcolors.SymLogNorm(linthresh=1, linscale=1, vmin=-0.1, vmax=0.1)
# DR_vs_2D_l55_LAT = DR_vertslice_2D_lat.sel(n_scales=0)
lat=DR_vertslice_2D_lat['latitude']
lev = DR_vertslice_2D_lat['level']

latw = w_vertslice_2D_lat['lat']
levw = w_vertslice_2D_lat['pressure_level']

LAT, LEV = np.meshgrid(latw, levw)
wxg = w_vertslice_2D_lat*10
vxg = v_vertslice_2D_lat/10
step=4
fig,ax=plt.subplots(figsize=(8,4))
CS = plt.contourf(lat,lev,-DR_vertslice_2D_lat.T,cmap='bwr',extend='both',levels=np.linspace(-0.1,0.1,101))
plt.quiver(LAT[:,::step], LEV[:,::step], vxg[:,::step], wxg[:,::step], scale=5, color='k',headwidth=7, headaxislength=2, headlength=5, width=0.003)  # Adjust 'scale' as needed
plt.contour(latw,levw,u_vertslice_2D_lat,colors='k',levels=[0],linewidths=2)
plt.contour(latw,levw,u_vertslice_2D_lat,colors='k',levels=np.linspace(-10,10,11),linewidths=1)

plt.gca().invert_yaxis() # The data is sorted by level index (index 12 = 1000hPa)
ax.set_xlim([-25,23])
plt.ylabel('Height (hPa)')
plt.xlabel(r'Latitude ($^\circ$N)')
# # plt.xticks([])
plt.title(r'$\ell$ = 220 km | DYAMOND Summer | ERA5 | u c.i. 2 m/s')

cbar = plt.colorbar(CS)
cbar.set_label(r'Longitudinally averaged $D_\ell(\mathbf{u})$ (m$^2$s$^{-3}$)')
# cbar.set_ticks(np.linspace(-0.1,0.1,11))
# cbar.formatter = ticker.ScalarFormatter()
# cbar.formatter.set_scientific(True)
# cbar.formatter.set_powerlimits((-3, 3))
# cbar.update_ticks()
cbar.set_ticks([-0.1,0,0.1])

plt.savefig('/home/users/emg97/emgPlots/LO_verticalslice_LAT_l220km_ERA5_vwquiv_ucontour.png',dpi=600,bbox_inches='tight')

##### Useful snippets

In [ ]:
directory = '/gws/nopw/j04/kscale/USERS/emg/data/LoSSET_out/'
model = 'channel_n2560_RAL3p2'
fname = f'{directory}/DRdir2dt_Nlmax5_{model}_DS_tmean.nc'
if not os.path.exists(fname):
    DR_tm = ds['LoSSET_DR'].mean(dim='time').compute()
    DR_tm.to_netcdf(fname)
DR_tm = xr.open_dataset(fname, mask_and_scale=True)['LoSSET_DR']
DR_tm